In [23]:
import cv2
import mediapipe as mp
import time
import numpy as np
import pyautogui

mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
    
def findHands(hands, image):
    return hands.process(image)
        
def drawHands(image, my_hands):
    global mp_hands
    if my_hands.multi_hand_landmarks:
        for hand_landmarks in my_hands.multi_hand_landmarks:
            mp.solutions.drawing_utils.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)            
    return image

def getPointer(image, my_hands):  # image is only used for its shape
    if my_hands.multi_hand_landmarks:
        for hand_landmarks in my_hands.multi_hand_landmarks:
            # check if landmark is tip of index
            if hand_landmarks.landmark[8].y < hand_landmarks.landmark[5].y:
                pos_idx = (int(hand_landmarks.landmark[8].x*image.shape[1]), int(hand_landmarks.landmark[8].y*image.shape[0]))
            else:
                pos_idx = (int(hand_landmarks.landmark[8].x*image.shape[1]), int(hand_landmarks.landmark[8].y*image.shape[0]))
            # check if landmark is tip of thumb
            if hand_landmarks.landmark[4].x < hand_landmarks.landmark[3].x:
                pos_thumb = (int(hand_landmarks.landmark[4].x*image.shape[1]), int(hand_landmarks.landmark[4].y*image.shape[0]))
            else:
                pos_thumb = (int(hand_landmarks.landmark[4].x*image.shape[1]), int(hand_landmarks.landmark[4].y*image.shape[0]))
    return pos_idx, pos_thumb

def getKeypoints(self):
    data = []
    if self.results.multi_hand_landmarks:
        for hand_landmarks in self.results.multi_hand_landmarks:
            for i, landmark in enumerate(hand_landmarks.landmark):
                data.append([landmark.x, landmark.y, landmark.z])
    data = np.array(data)
    
    hands = []
    centers = []
    for i in range(0, data.shape[0], 21):
        hands.append(data[i:i+21])
        centers.append(np.mean(hands[-1], axis = 0).reshape(3))
    hands = np.array(hands).reshape(-1, 21, 3)
    centers = np.array(centers).reshape(-1, 3)
        
    if len(centers.shape) == 2 and centers.shape[0] >= 2:
        dist = (np.sum((centers[0, :2] - centers[1, :2])**2))**0.5
    else:
        dist = None
    
    dataset = {
        "hands": hands,
        "centers": centers,
        "distance": dist
    }
    return dataset


cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920//4)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080//4)
dataset = []

prev = time.time()
history = []


In [28]:

while True:
    # Read frame, make it rgb and flip
    ret, frame = cap.read()
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = cv2.flip(image, 1)

    if image.mean() < 90:
        break

    # Find keypoints, draw them and get back useful data
    t0 = time.time()
    myhands = findHands(hands, image)
    drawHands(image, myhands)
    data = getKeypoints(myhands)
    pol, ind = getPointer(image, myhands)

    print(time.time()-t0)

    
    if pol is not None and ind is not None:
        mean = (pol[0]+ind[0])/2, (pol[1]+ind[1])/2 
        dist = ((pol[0]-ind[0])**2 + (pol[1]-ind[1])**2)**0.5

        if dist < 50:
            cv2.circle(image, (int(mean[0]), int(mean[1])), 10, (255, 255, 255), -1)
            pyautogui.moveTo(mean[0]*4, mean[1]*4)
            pyautogui.click()

        elif dist > 50 and dist < 250:
            cv2.circle(image, (int(mean[0]), int(mean[1])), 10, (0, 0, 0), -1)
            pyautogui.moveTo(mean[0]*4, mean[1]*4)
            # print(mean[0]*4, mean[1]*4)

    
    # Reshape bgr image
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    image = cv2.resize(image, (1280, 720))
    
    # find centers
    if len(data["centers"]) > 0:
        for x, y, z in data["centers"]:
            image = cv2.circle(image, (int(x*1280), int(y*720)), 10, (255, 255, 255), -1)


    tuttiPugni = True
    n = 0
    

    
    cv2.putText(image, str(round(1/(time.time()-prev), 1)), (20, 50+50*(n+2)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    prev = time.time()
    
    cv2.imshow('Hand Tracking', cv2.resize(image, (1600//2, 900//2)))

    # Step 10: Break the loop if the user presses the 'q' key
    if cv2.waitKey(10) & (0xFF == ord('q') or 0xFF == 27):
        break    

    
    
    


0.0431981086730957
0.04055190086364746
0.06732749938964844
0.06760072708129883
0.09799957275390625
0.0644979476928711
0.032781362533569336
0.10554265975952148
0.06856608390808105
0.06254076957702637
0.07964324951171875
0.10613727569580078
0.09882593154907227
0.10156440734863281
0.10955810546875
0.08994483947753906
0.08863520622253418
0.08363866806030273
0.07962584495544434


In [6]:



# Step 9: Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()

